### ingest  private-gpt  解析

```python
#### components/ingest_helper.py
# IngestionHelper  加载files，-> [Document] filter-node-metadata
"""  
BaseNode (抽象基类)
 ├─ TextNode
 │    └─ Document   ← 你最常用/读写的入口（整份文件或一页/一段）
 ├─ ImageNode
 ├─ IndexNode
 └─ ...（其他模态/复合节点）

- BaseNode：所有“可被索引/检索”的最小通用接口。
- TextNode：带 text 的文本节点。
- Document：实质上就是一个“整份文档/页”的 TextNode（许多版本里 Document 直接继承 TextNode）。读文件的 reader 通常返回 list[Document] 作为初始输入。
-  IngestionHelper.transform_file_into_documents() 里做的事：把各种文件读成 List[Document]，再做 metadata 规范化与清洗。


- BaseNode：统一抽象（ID、metadata、关系、可被索引/检索/持久化）。
- Document：读入阶段的“整文/整页文本节点”，是最自然的入口；从它向下切块得到更多 TextNode。
- Node（TextNode 等）：最小检索单元，被向量化、被重排、被拼成上下文回给 LLM。

# BaseNode 是什么（共同的骨架）
任何节点（文本、图片、索引引用…）都具备这些“基因”——常见属性/行为（名字可能随版本有细微差别，但概念稳定）：
id_：节点的唯一 ID（一个 chunk 一条 ID）。
metadata: dict：任意键值对（如 file_name, page_label, source 等）。
relationships：前后文/层级联系（如 PREVIOUS/NEXT/PARENT/SOURCE 等关系，用于重建上下文或来源）。
hash：内容哈希，便于幂等/变更检测。
excluded_embed_metadata_keys：做向量化时不带入的 metadata（减少噪声）。
excluded_llm_metadata_keys：喂给 LLM 上下文时不带入的 metadata（避免 prompt 污染）。
（有的版本）ref_doc_id 或通过关系把节点映射回原始文档，用于 delete_ref_doc 之类操作。
作用：统一“索引/存储/检索”的数据接口——向量库只关心它的文本与 embedding、LLM 只关心它的文本与必要 metadata、文档层级/前后窗口用 relationships 还原。

# TextNode：带文本的通用节点
在 BaseNode 基础上，TextNode提供：
text: str：节点文本（chunk 的正文）。
（常见）start_char_idx/end_char_idx：文本在原文中的切片位置（用于追踪来源、展示命中高亮）。
其他用于格式化/模板的字段（不同版本略有差异）。
作用：凡是“可被切块后的文本单元”，基本就是 TextNode（或其子类）。


# Document：入口文档，也是文本节点
Document 是 TextNode 的一个“角色化”版本，语义是“一份原始文档（或 PDF 的一页、或 HTML 的一个段落）”。在多数 Reader 里，读取一个文件会返回若干个 Document（PDF 常常是“每页一个 Document”）。
为什么工程里用它当入口？
Reader 输出：reader.load_data(path) → list[Document]
携带原始语义：你可以在 metadata 放 file_name, page_label, source, mtime 等，后面切块时会自动继承。
支持删除/追踪：很多删除操作（delete_ref_doc）是按“文档维度”生效的：你把文档的 doc_id 记录到每个 chunk（node）的 ref_doc 关系里，后面就能通过 doc_id 一键删掉这个文档对应的所有节点与向量。

document.metadata["file_name"] = file_name
document.metadata["doc_id"] = document.doc_id
document.excluded_embed_metadata_keys = ["doc_id"]
document.excluded_llm_metadata_keys = ["file_name", "doc_id", "page_label"]
给 Document 附上来源；
用 doc_id 当“文档级别的主键”，供后续删除/查询；
设置 excluded_*：避免把无关 metadata 污染 embedding/LLM。

"""

# components/ingest_component.py
"""
interface:
# llama_index.core.schema
    - Document：文本文档节点（继承自 TextNode/Node）；reader 输出的基本单位；携带 text/metadata/doc_id。
    - BaseNode：所有节点的抽象基类；insert_nodes() 接口处理的对象类型。
    - TransformComponent：变换组件协议（如 splitter/embedding）；run_transformations 的元素类型。
# llama_index.core.ingestion
    - run_transformations(documents, transformations, ...)：按顺序执行一串变换（典型是 [node_parser/splitter, embedder]），将 Document 变成带向量的 BaseNode 列表。
# llama_index.core.indices
    - VectorStoreIndex：RAG 常用的索引类型；支持 from_documents()、insert(document)、insert_nodes(nodes)、delete_ref_doc(doc_id, ...)。
    - load_index_from_storage(storage_context, ...)：从持久化存储中加载索引（与 StorageContext 配合）。
# llama_index.core.indices.base
    - BaseIndex：索引抽象基类（这里主要用作类型注解）。
# llama_index.core.data_structs
    - IndexDict：某些索引的内部数据结构类型参数（用于 BaseIndex[IndexDict] 的类型标注）。
# llama_index.core.storage
    - StorageContext：封装 vector_store / docstore / index_store 的上下文；负责持久化、加载、协调三者。
# llama_index.core.embeddings.utils
    - EmbedType：对“可用作嵌入器”的类型别名（抽象成一个联合/协议），便于函数签名与类型提示。
工程内自有部分
# IngestionHelper：文件→Documents 的解析与元数据规范化。
# local_data_path：存储持久化位置。
# Settings：工程配置（决定 ingest_mode / workers 等）。





"""



# settings.py





```

#### 按需import的方法

哪个先导入成功就立刻返回对应的类；只有当所有路径都失败时才抛出最后一次的异常。 (吞掉所有的异常)

tgt_clss = _import(["from a import tgt_clss",
                    "from a.sub import tgt_clss",
                    "from a.core import tgt_clss",
                    "from a.new import tgt_clss"])


```python
# ---------- 工厂：延迟导入，适配不同版本的 llama-index ----------
def _import(path_variants: list[str]):
    """尝试多条导入路径，适配 llama-index 不同版本目录结构。"""
    last_err = None
    for p in path_variants:
        try:
            module_path, name = p.rsplit(".", 1)
            mod = __import__(module_path, fromlist=[name])
            return getattr(mod, name)
        except Exception as e:  # pragma: no cover
            last_err = e
    raise last_err
``` 

优化版本，有个error-log
tgt_clss = safe_import(["a.tgt_clss",
                        "a.sub.tgt_clss",
                        "a.core.tgt_clss",
                        "a.new.tgt_clss"])

```python
from importlib import import_module

def safe_import(path_variants: list[str]):
    errors = []
    for p in path_variants:
        try:
            module_path, name = p.rsplit(".", 1)
            mod = import_module(module_path)
            return getattr(mod, name)
        except (ImportError, ModuleNotFoundError, AttributeError) as e:
            errors.append(f"{p} -> {e}")
            continue
    raise ImportError("All import variants failed:\n" + "\n".join(errors))
```

#### 注入器：  
- `cfg.yaml --- 通过接口更新 ---> Settings ---注入器--->  工程代码实例`
- `对于一个具体的 Settings  --- 调用 --> 实例化  --> 工程代码  就足够`


##### llama-index official-reranker-interface: class`SentenceTransformerRerank`
```python
from llama_index.core.postprocessor import SentenceTransformerRerank
```

In [1]:
####  reranker  

# official-api: 
from llama_index.core.postprocessor import SentenceTransformerRerank
reranker = SentenceTransformerRerank(
    model="Qwen/Qwen3-Reranker-0.6B",
    top_n=3,
    device="cpu",
    trust_remote_code=True,
    keep_retrieval_score=False,
)   
# DEFAULT_SENTENCE_TRANSFORMER_MAX_LENGTH = 512

""" 
# sbert_rerank.py
from typing import Any, List, Optional

from llama_index.core.bridge.pydantic import Field, PrivateAttr
from llama_index.core.callbacks import CBEventType, EventPayload
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import MetadataMode, NodeWithScore, QueryBundle
from llama_index.core.utils import infer_torch_device

DEFAULT_SENTENCE_TRANSFORMER_MAX_LENGTH = 512


class SentenceTransformerRerank(BaseNodePostprocessor):
    model: str = Field(description="Sentence transformer model name.")
    top_n: int = Field(description="Number of nodes to return sorted by score.")
    device: str = Field(
        default="cpu",
        description="Device to use for sentence transformer.",
    )
    keep_retrieval_score: bool = Field(
        default=False,
        description="Whether to keep the retrieval score in metadata.",
    )
    trust_remote_code: bool = Field(
        default=False,
        description="Whether to trust remote code.",
    )
    _model: Any = PrivateAttr()

    def __init__(
        self,
        top_n: int = 2,
        model: str = "cross-encoder/stsb-distilroberta-base",
        device: Optional[str] = None,
        keep_retrieval_score: bool = False,
        trust_remote_code: bool = True,
    ):
        try:
            from sentence_transformers import CrossEncoder  # pants: no-infer-dep
        except ImportError:
            raise ImportError(
                "Cannot import sentence-transformers or torch package,",
                "please `pip install torch sentence-transformers`",
            )
        device = infer_torch_device() if device is None else device
        super().__init__(
            top_n=top_n,
            model=model,
            device=device,
            keep_retrieval_score=keep_retrieval_score,
        )
        self._model = CrossEncoder(
            model,
            max_length=DEFAULT_SENTENCE_TRANSFORMER_MAX_LENGTH,
            device=device,
            trust_remote_code=trust_remote_code,
        )

    @classmethod
    def class_name(cls) -> str:
        return "SentenceTransformerRerank"

    def _postprocess_nodes(
        self,
        nodes: List[NodeWithScore],
        query_bundle: Optional[QueryBundle] = None,
    ) -> List[NodeWithScore]:
        if query_bundle is None:
            raise ValueError("Missing query bundle in extra info.")
        if len(nodes) == 0:
            return []

        query_and_nodes = [
            (
                query_bundle.query_str,
                node.node.get_content(metadata_mode=MetadataMode.EMBED),
            )
            for node in nodes
        ]

        with self.callback_manager.event(
            CBEventType.RERANKING,
            payload={
                EventPayload.NODES: nodes,
                EventPayload.MODEL_NAME: self.model,
                EventPayload.QUERY_STR: query_bundle.query_str,
                EventPayload.TOP_K: self.top_n,
            },
        ) as event:
            scores = self._model.predict(query_and_nodes)

            assert len(scores) == len(nodes)

            for node, score in zip(nodes, scores):
                if self.keep_retrieval_score:
                    # keep the retrieval score in metadata
                    node.node.metadata["retrieval_score"] = node.score
                node.score = score

            new_nodes = sorted(nodes, key=lambda x: -x.score if x.score else 0)[
                : self.top_n
            ]
            event.on_end(payload={EventPayload.NODES: new_nodes})

        return new_nodes


"""

"""  
# Reranker在什么时候有用？ 
- 常见做法是 先 用向量检索（bi-encoder embedding）拿到 top_k 候选（比如 50～200 条），再 用 cross-encoder 做 rerank，把前 top_n（比如 5～10）按相关性精排输出。
- 为什么通常能提升效果：cross-encoder 每次把「查询 + 文本」一起喂给模型，做交互式编码，能捕捉词序、约束关系、否定词等细节，P@k 和 nDCG 往往显著提升，尤其是长句/多条件/歧义查询。
- 什么时候“感觉不太有用”：
    - similarity_top_k 太小（前段召回没把真相关召回来），reranker 再“精排”也白搭。
    - 文本很短/关键词检索足够（如 FAQ 单句），embedding 已经很稳
    - top_n 设太小 → 损失召回。一般建议：先召回 50～200，再 rerank 到 5～10。
- 代价：cross-encoder 需要对每个候选“成对”打分，比单纯向量检索慢很多；长文本还受最大长度限制



# 不同 reranker 实现“流派”
- Pair-wise CrossEncoder（本类）：对每个 (q, d) 单独打分，稳定、常用。代表：BAAI/bge-reranker-*、jinaai/jina-reranker-* 等。
- List-wise Reranker：一次看全体候选，建模文档间相对顺序，质量可更高，但接口不同、显存开销大。
- LLM Reranker（生成式/打分式）：用小/中型 LLM 读 “查询+候选文本”，按模板让它“给分/排序”。在 LlamaIndex 里通常用 LLM 的 postprocessor（例如 LLMRerank、自定义 BaseNodePostprocessor），不是 SentenceTransformerRerank。
- 托管 API：如 Cohere/Vectara 等，也有各自的 LlamaIndex 后处理器（CohereRerank 等）。

"""



Some weights of Qwen3ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen3-Reranker-0.6B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import span_marker 



